In [4]:
import pandas as pd

In [5]:
data_dir = '/kaggle/input/otto-train-and-test-data-for-local-validation'

df_train = pd.read_parquet(f'{data_dir}/train.parquet')
df_val = pd.read_parquet(f'{data_dir}/test.parquet')

In [6]:
df_val_part = df_val.iloc[:10]
df_val_part.columns

Index(['session', 'aid', 'ts', 'type'], dtype='object')

In [26]:
# step1
## sessionでまとめたデータを入力し、candidateとなるaidのlist返却
def generate_candidate(k = 50):
    # 入力 ある

    return [1, 2, 3, 4]


In [27]:
df_val

,session,aid,ts,type
0,11098528,11830,1661119200,0
1,11098529,1105029,1661119200,0
2,11098530,264500,1661119200,0
3,11098530,264500,1661119288,0
4,11098530,409236,1661119369,0
...,...,...,...,...
7683572,12899774,33035,1661723968,0
7683573,12899775,1743151,1661723970,0
7683574,12899776,548599,1661723972,0
7683575,12899777,384045,1661723976,0


In [31]:
candidates = df_val.groupby('session').apply(generate_candidate).explode()


In [32]:
candidates.name = 'item'
candidates = candidates.to_frame().reset_index()
candidates

,session,item
0,11098528,1
1,11098528,2
2,11098528,3
3,11098528,4
4,11098529,1
...,...,...
7204999,12899777,4
7205000,12899778,1
7205001,12899778,2
7205002,12899778,3


In [33]:
# step2
item_features = pd.merge(df_train, df_val).groupby('aid').agg({'aid': 'count', 'session': 'nunique', 'type': 'mean'})
item_features.columns = ['item_item_count', 'item_user_count', 'item_by_count']
item_features.to_parquet('item_features.pqt')

In [36]:
# step3
user_features = df_val.groupby('session').agg({'session': 'count', 'aid': 'nunique', 'type': 'mean'})
user_features.columns = ['user_user_count','user_item_count','user_buy_ratio']
# CONVERT COLUMNS TO INT32 and FLOAT32 HERE
user_features.to_parquet('user_features.pqt')

In [ ]:
# step4


In [32]:
# step5
item_features = pd.read_parquet('item_features.pqt')
candidates = candidates.merge(item_features, left_on='item', right_index=True, how='left').fillna(-1)
user_features = pd.read_parquet('user_features.pqt')
candidates = candidates.merge(user_features, left_on='user', right_index=True, how='left').fillna(-1)

In [33]:
candidates

,session,aid,item_item_count,item_user_count,item_by_count,user_user_count,user_item_count,user_buy_ratio
0,11098528,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,11098528,2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,11098528,3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,11098528,4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,11098529,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...
7204999,12899777,4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7205000,12899778,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7205001,12899778,2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7205002,12899778,3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [48]:
tar = pd.read_parquet(f'{data_dir}/test_labels.parquet')
tar = tar.loc[ tar['type']=='clicks' ]
tar['item'] = tar['ground_truth'].explode().astype('int32')
tar['click'] = 1

In [49]:
tar = tar[['user', 'item', 'click']]
tar

,session,user,click
0,11098528,1679529,1
3,11098529,1105029,1
6,11098532,1596491,1
7,11098533,1417450,1
10,11098534,908024,1
...,...,...,...
2212687,12899774,1399483,1
2212688,12899775,1760714,1
2212689,12899776,1737908,1
2212690,12899777,384045,1


In [52]:
candidates = candidates.merge(tar,on=['user','item'],how='left').fillna(0)

KeyError: 'user'

In [50]:
import xgboost as xgb
from sklearn.model_selection import GroupKFold

skf = GroupKFold(n_splits=5)
for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, candidates['click'], groups=candidates['user'] )):

    X_train = candidates.loc[train_idx, FEATURES]
    y_train = candidates.loc[train_idx, 'click']
    X_valid = candidates.loc[valid_idx, FEATURES]
    y_valid = candidates.loc[valid_idx, 'click']

    # IF YOU HAVE 50 CANDIDATE WE USE 50 BELOW
    dtrain = xgb.DMatrix(X_train, y_train, group=[50] * (len(train_idx)//50) ) 
    dvalid = xgb.DMatrix(X_valid, y_valid, group=[50] * (len(valid_idx)//50) ) 

    xgb_parms = {'objective':'rank:pairwise', 'tree_method':'gpu_hist'}
    model = xgb.train(xgb_parms, 
        dtrain=dtrain,
        evals=[(dtrain,'train'),(dvalid,'valid')],
        num_boost_round=1000,
        verbose_eval=100)
    model.save_model(f'XGB_fold{fold}_click.xgb')

KeyError: 'click'